TIF images → `Extract-Channels` → `Visualize-Inputs` → `Predict-Segments` → `Visualize-Segmentation` → predicted segmentations

# Set up Mesmer + inputs

In [1]:
import os

In [2]:
# Parameters (overridden by notebook parameters)
input_channels_path = './saves/input_channels.npz'
model_path = os.path.expanduser('~') + '/.keras/models/MultiplexSegmentation'
output_path = './saves'

In [3]:
from deepcell.applications import Mesmer
import tensorflow as tf

model = tf.keras.models.load_model(model_path)

app = Mesmer(model=model)

2023-07-10 15:25:31.047780: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import numpy as np

with np.load(input_channels_path) as loader:
    # An array of shape [num_inputs, height, width] containing intensity of nuclear channel
    inputs_nuclear = loader['nuclear']

    # An array of shape [num_inputs, height, width] containing intensity of membrane channel
    inputs_membrane = loader['membrane']

# An array of shape [num_inputs, height, width, 2] containing nuclear & membrane channels
input_channels = np.stack((inputs_nuclear, inputs_membrane), axis=-1)

# Generate predictions

This step is expensive!

In [5]:
segmentation_predictions = app.predict(input_channels, image_mpp=0.5)

# Save predictions

In [6]:
import numpy as np
np.savez_compressed(output_path + '/segmentation_predictions.npz', predictions=segmentation_predictions)